In [1]:
# SeqRecon-AD-S: 有监督异常检测（位置感知 Transformer + 二分类头，骨干与 SeqRecon-AD 一致）

import math
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, f1_score, precision_score, recall_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

Using device: cuda


In [2]:
# 1. 数据加载与预处理（与 SeqRecon-AD 一致）

card_item = pd.read_csv('card_item.csv')
card_feats = pd.read_csv('card_feats.csv', usecols=['label','card_id','name','身份证号','age'])
dataset = pd.concat([card_item, card_feats], axis=1)

import ast
if isinstance(dataset['明细项目名称'].iloc[0], str):
    dataset['明细项目名称'] = dataset['明细项目名称'].apply(ast.literal_eval)

print('数据规模:', len(dataset))

with open('item2id.json', 'r', encoding='utf-8') as f:
    item2id = json.load(f)
id2item = {v: k for k, v in item2id.items()}
num_items = len(item2id)

def map_items_to_ids(items, item2id):
    return [item2id[item] for item in items if item in item2id]

dataset['明细项目ID'] = dataset['明细项目名称'].apply(lambda x: map_items_to_ids(x, item2id))

train_df, temp_df = train_test_split(dataset, test_size=0.2, random_state=42)
val_df, test_df   = train_test_split(temp_df, test_size=0.5, random_state=42)
print(f"train={len(train_df)}, val={len(val_df)}, test={len(test_df)}")

数据规模: 8917
train=7133, val=892, test=892


In [3]:
# 2. 位置感知 Transformer 骨干 + 二分类头 + next-item 头（多任务 L=L_sup+λ*L_next）

class RelativePositionalEncoding(nn.Module):
    def __init__(self, num_heads, max_len=512):
        super().__init__()
        self.rel_pos_table = nn.Parameter(torch.randn(2 * max_len - 1, num_heads))
        self.max_len = max_len

    def forward(self, q_len, k_len):
        range_q = torch.arange(q_len)[:, None]
        range_k = torch.arange(k_len)[None, :]
        distance_mat = range_k - range_q
        distance_mat = distance_mat.clamp(-self.max_len + 1, self.max_len - 1)
        distance_mat += self.max_len - 1
        rel_bias = self.rel_pos_table[distance_mat]
        rel_bias = rel_bias.permute(2, 0, 1)
        return rel_bias


class RelativeMultiheadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.2):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout = dropout
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value, attn_mask=None, key_padding_mask=None, pos_bias=None):
        B, L, D = query.shape
        H = self.num_heads
        d = D // H
        q = self.q_proj(query).view(B, L, H, d).transpose(1, 2)
        k = self.k_proj(key).view(B, L, H, d).transpose(1, 2)
        v = self.v_proj(value).view(B, L, H, d).transpose(1, 2)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d)
        if pos_bias is not None:
            scores = scores + pos_bias.unsqueeze(0)
        if attn_mask is not None:
            scores = scores + attn_mask.unsqueeze(0).unsqueeze(0)
        if key_padding_mask is not None:
            scores = scores.masked_fill(key_padding_mask.unsqueeze(1).unsqueeze(2), float('-inf'))
        attn_weights = torch.softmax(scores, dim=-1)
        attn_weights = F.dropout(attn_weights, p=self.dropout, training=self.training)
        attn_output = torch.matmul(attn_weights, v)
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, L, D)
        return self.out_proj(attn_output)


class CustomTransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.2):
        super().__init__()
        self.self_attn = RelativeMultiheadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, d_model * 4)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_model * 4, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None, pos_bias=None):
        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask, pos_bias=pos_bias)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src


class TransformerAnomalyDetectorSupervised(nn.Module):
    """骨干与 SeqRecon-AD 一致，二分类头 + next-item 头；多任务 L=L_sup+λ*L_next"""
    def __init__(self, embedding_matrix, d_model=512, nhead=4, num_layers=6, dropout=0.2, pad_idx=0):
        super().__init__()
        self.d_model = d_model
        self.pad_idx = pad_idx

        num_items, embedding_dim = embedding_matrix.size()
        embedding_matrix = (embedding_matrix - embedding_matrix.mean()) / (embedding_matrix.std() + 1e-8)

        self.embedding = nn.Embedding(num_items, embedding_dim)
        self.embedding.weight = nn.Parameter(embedding_matrix)
        self.embedding.weight.requires_grad = False

        self.embed_proj = nn.Linear(embedding_dim, d_model)
        self.pos_encoder = RelativePositionalEncoding(num_heads=nhead, max_len=512)
        self.layers = nn.ModuleList([
            CustomTransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
            for _ in range(num_layers)
        ])
        self.final_norm = nn.LayerNorm(d_model)
        self.predictor = nn.Linear(d_model, num_items)
        self.cls_head = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, 2),
        )
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embed_proj.bias.data.zero_()
        self.embed_proj.weight.data.uniform_(-initrange, initrange)
        nn.init.xavier_uniform_(self.predictor.weight)
        self.predictor.bias.data.zero_()
        for m in self.cls_head:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()

    def generate_mask(self, seq_len, device):
        mask = (torch.triu(torch.ones(seq_len, seq_len)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, 0.0)
        return mask.to(device)

    def forward(self, src, src_mask=None):
        B, L = src.size()
        src_emb = self.embedding(src)
        src_emb = self.embed_proj(src_emb) * math.sqrt(self.d_model)
        src_emb = F.layer_norm(src_emb, src_emb.shape[-1:])

        pos_bias = self.pos_encoder(L, L)
        src_key_padding_mask = (src == self.pad_idx)

        output = src_emb
        for layer in self.layers:
            output = layer(output, src_mask=src_mask,
                           src_key_padding_mask=src_key_padding_mask,
                           pos_bias=pos_bias)
        output = self.final_norm(output)

        predictions = self.predictor(output)
        seq_repr = (output * (~src_key_padding_mask).unsqueeze(-1)).sum(1) / (
            (~src_key_padding_mask).sum(1, keepdim=True).float().clamp(min=1)
        )
        cls_logits = self.cls_head(seq_repr)
        return predictions, cls_logits

    def compute_loss_cls(self, src, labels, mask=None):
        _, cls_logits = self.forward(src)
        labels = labels.long().clamp(0, 1)
        return F.cross_entropy(cls_logits, labels)

    def compute_loss_next(self, src, tgt, mask=None):
        seq_len = src.size(1)
        causal_mask = self.generate_mask(seq_len, src.device)
        predictions, _ = self.forward(src, src_mask=causal_mask)
        predictions = predictions[:, :-1, :].contiguous()
        tgt = tgt[:, 1:].contiguous()
        if mask is not None:
            mask = mask[:, 1:].contiguous()
            active = mask.view(-1) == 1
            loss = F.cross_entropy(
                predictions.view(-1, predictions.size(-1)), tgt.view(-1), reduction='none'
            )
            loss = loss[active].mean()
        else:
            loss = F.cross_entropy(
                predictions.view(-1, predictions.size(-1)), tgt.view(-1), ignore_index=self.pad_idx
            )
        if torch.isnan(loss) or torch.isinf(loss):
            return torch.tensor(0.0, device=src.device, dtype=loss.dtype)
        return loss

In [4]:
# 3. Dataset / DataLoader（与 SeqRecon-AD 一致）

class PrescriptionDataset(Dataset):
    def __init__(self, dataframe, max_length=517, pad_idx=0):
        self.data = dataframe.reset_index(drop=True)
        self.max_length = max_length
        self.pad_idx = pad_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence = self.data.iloc[idx]['明细项目ID']
        if len(sequence) > self.max_length:
            sequence = sequence[:self.max_length]
            original_len = self.max_length
        else:
            original_len = len(sequence)
            sequence = sequence + [self.pad_idx] * (self.max_length - len(sequence))
        input_seq = target_seq = sequence
        mask = [1] * original_len + [0] * (self.max_length - original_len)
        label = self.data.iloc[idx]['label']
        return {
            'input_seq': torch.tensor(input_seq, dtype=torch.long),
            'target_seq': torch.tensor(target_seq, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.float),
            'label': torch.tensor(label, dtype=torch.float),
        }


def custom_collate_fn(batch):
    input_seqs = torch.stack([b['input_seq'] for b in batch])
    target_seqs = torch.stack([b['target_seq'] for b in batch])
    masks = torch.stack([b['mask'] for b in batch])
    labels = torch.stack([b['label'] for b in batch])
    return {'input_seq': input_seqs, 'target_seq': target_seqs, 'mask': masks, 'label': labels}


train_dataset = PrescriptionDataset(train_df)
val_dataset   = PrescriptionDataset(val_df)
test_dataset  = PrescriptionDataset(test_df)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  collate_fn=custom_collate_fn)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

In [5]:
# 4. 评估函数：分类指标 + Recall@10/NDCG@10；F1 阈值可在验证集上优化

def evaluate_retrieval(model, data_loader, device, k=10):
    """Next-item 检索：Recall@k, NDCG@k"""
    model.eval()
    total_recall, total_ndcg, total_cnt = 0.0, 0.0, 0
    with torch.no_grad():
        for batch in data_loader:
            input_seq = batch['input_seq'].to(device)
            target_seq = batch['target_seq'].to(device)
            mask = batch['mask'].to(device)
            B, L = input_seq.size()
            causal_mask = model.generate_mask(L, device)
            logits, _ = model(input_seq, src_mask=causal_mask)
            last_logits = logits[:, -1, :].clone()
            last_logits[:, 0] = -float('inf')
            _, topk_indices = torch.topk(last_logits, k=k, dim=-1)
            lengths = mask.sum(dim=1).long()
            next_indices = (lengths - 1).clamp(min=0)
            next_item = target_seq.gather(1, next_indices.view(-1, 1)).squeeze(1)
            gt = next_item.cpu().numpy()
            pred = topk_indices.cpu().numpy()
            for g, p in zip(gt, pred):
                if np.any(p == g):
                    total_recall += 1.0
                    rank = np.where(p == g)[0][0] + 1
                    total_ndcg += 1.0 / math.log2(rank + 1)
                total_cnt += 1
    return total_recall / total_cnt, total_ndcg / total_cnt


def get_probs_and_labels(model, data_loader, device):
    """获取模型在 data_loader 上的异常类概率与真实标签（用于阈值搜索）"""
    model.eval()
    all_logits, all_labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_seq = batch['input_seq'].to(device)
            _, cls_logits = model(input_seq)
            all_logits.append(cls_logits.cpu().numpy())
            all_labels.append(batch['label'].numpy())
    probs = torch.softmax(torch.tensor(np.concatenate(all_logits)), dim=-1)[:, 1].numpy()
    labels = np.concatenate(all_labels)
    return probs, labels


def get_best_f1_threshold(probs, labels, num_steps=101):
    """在 [0,1] 上搜索使 F1 最大的阈值，与 SeqRecon-AD 等统一可比"""
    best_threshold, best_f1 = 0.5, 0.0
    best_precision, best_recall = 0.0, 0.0
    for i in range(num_steps):
        thresh = i / (num_steps - 1) if num_steps > 1 else 0.5
        pred = (probs >= thresh).astype(int)
        p = precision_score(labels, pred, zero_division=0)
        r = recall_score(labels, pred, zero_division=0)
        f1 = f1_score(labels, pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = thresh
            best_precision, best_recall = p, r
    return best_threshold, best_f1, best_precision, best_recall


def evaluate_cls(model, data_loader, device, threshold=0.5):
    """仅分类指标：AUC, PR-AUC, Precision, Recall, F1（有监督异常检测）。"""
    model.eval()
    all_cls_logits = []
    all_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_seq = batch['input_seq'].to(device)
            _, cls_logits = model(input_seq)
            all_cls_logits.append(cls_logits.cpu().numpy())
            all_labels.append(batch['label'].numpy())
    all_cls_logits = np.concatenate(all_cls_logits, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    probs = torch.softmax(torch.tensor(all_cls_logits), dim=-1)[:, 1].numpy()
    pred_labels = (probs >= threshold).astype(int)
    return {
        'AUC': roc_auc_score(all_labels, probs),
        'PR_AUC': auc(precision_recall_curve(all_labels, probs)[1], precision_recall_curve(all_labels, probs)[0]),
        'Precision': precision_score(all_labels, pred_labels, zero_division=0),
        'Recall': recall_score(all_labels, pred_labels, zero_division=0),
        'F1': f1_score(all_labels, pred_labels, zero_division=0),
        'threshold_used': threshold,
    }

In [6]:
# 5. 有监督训练循环：仅二分类损失（有监督异常检测），按验证集 F1 保存最佳

import copy

def train_supervised(model, train_loader, val_loader, device,
                     max_epochs=50, patience=5, lambda_next=0.0,
                     save_path='seqrecon_supervised_best.pt'):
    """L = L_sup + λ*L_next；lambda_next=0 即仅分类。"""
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.9)

    best_f1 = 0.0
    best_state = None
    no_improve = 0

    for epoch in range(1, max_epochs + 1):
        model.train()
        total_loss = 0.0
        total_batches = 0

        for batch in tqdm(train_loader, desc=f'Epoch {epoch}/{max_epochs}'):
            input_seq = batch['input_seq'].to(device)
            target_seq = batch['target_seq'].to(device)
            mask = batch['mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            L_sup = model.compute_loss_cls(input_seq, labels, mask)
            L_next = model.compute_loss_next(input_seq, target_seq, mask) if lambda_next != 0 else torch.tensor(0.0, device=device)
            loss = L_sup + lambda_next * L_next
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            total_batches += 1

        scheduler.step()
        avg_loss = total_loss / total_batches

        model.eval()
        val_probs, val_labels = get_probs_and_labels(model, val_loader, device)
        best_thresh, best_f1_at_thresh, _, _ = get_best_f1_threshold(val_probs, val_labels)
        val_metrics = evaluate_cls(model, val_loader, device, threshold=0.5)

        print(f'Epoch {epoch} Completed, Avg Loss: {avg_loss:.4f}')
        print(f'Val AUC: {val_metrics["AUC"]:.4f}, PR-AUC: {val_metrics["PR_AUC"]:.4f}, '
              f'F1@0.5: {val_metrics["F1"]:.4f} | F1@best: {best_f1_at_thresh:.4f} (thr={best_thresh:.3f})')

        if best_f1_at_thresh > best_f1:
            best_f1 = best_f1_at_thresh
            best_state = copy.deepcopy(model.state_dict())
            torch.save(best_state, save_path)
            print('Saved new best model (by F1 at optimal threshold)')
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return model

In [ ]:
# 8. 多任务权重 λ 实验：L = L_sup + λ*L_next，分析 next-item 辅助任务对弱监督检测性能的影响
# λ ∈ {0, 0.1, 0.3, 0.5, 0.8, 1.0}，骨干与训练配置固定

import gc

LAMBDA_VALUES = [0.8,1.5]
max_epochs = 150
patience = 10
results_lambda = []

embedding_dim = 4096
pretrained_emb = nn.Embedding(num_items, embedding_dim)
pretrained_emb.load_state_dict(torch.load('item_embedding.pt'))
with torch.no_grad():
    embedding_matrix = pretrained_emb.weight.clone().detach()

for lam in LAMBDA_VALUES:
    print(f'\n========== λ = {lam} ==========')
    save_path = f'seqrecon_supervised_lambda_{lam}.pt'
    model_lam = TransformerAnomalyDetectorSupervised(
        embedding_matrix=embedding_matrix,
        d_model=512, nhead=2, num_layers=8, dropout=0.1, pad_idx=0,
    ).to(device)
    model_lam = train_supervised(
        model_lam,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        max_epochs=max_epochs,
        patience=patience,
        lambda_next=lam,
        save_path=save_path,
    )
    model_lam.load_state_dict(torch.load(save_path))
    model_lam.eval()
    # Avg_Loss (L_sup + λ*L_next) on test
    test_loss_sum, test_n = 0.0, 0
    with torch.no_grad():
        for batch in test_loader:
            inp = batch['input_seq'].to(device)
            tgt = batch['target_seq'].to(device)
            m = batch['mask'].to(device)
            lab = batch['label'].to(device)
            L_sup = model_lam.compute_loss_cls(inp, lab, m)
            L_next = model_lam.compute_loss_next(inp, tgt, m)
            test_loss_sum += (L_sup.item() + lam * L_next.item()) * inp.size(0)
            test_n += inp.size(0)
    avg_loss = test_loss_sum / max(test_n, 1)
    recall10, ndcg10 = evaluate_retrieval(model_lam, test_loader, device, k=10)
    val_probs, val_labels = get_probs_and_labels(model_lam, val_loader, device)
    best_threshold, best_f1_val, _, _ = get_best_f1_threshold(val_probs, val_labels)
    test_metrics = evaluate_cls(model_lam, test_loader, device, threshold=best_threshold)
    results_lambda.append({
        'lambda': lam,
        'Avg_Loss': round(avg_loss, 4),
        'Recall@10': round(recall10, 4),
        'NDCG@10': round(ndcg10, 4),
        'AUC': round(test_metrics['AUC'], 4),
        'PR-AUC': round(test_metrics['PR_AUC'], 4),
        'F1': round(test_metrics['F1'], 4),
        'Precision': round(test_metrics['Precision'], 4),
        'Recall': round(test_metrics['Recall'], 4),
    })
    print(results_lambda)
    del model_lam
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

cols = ['lambda', 'Avg_Loss', 'Recall@10', 'NDCG@10', 'AUC', 'PR-AUC', 'F1', 'Precision', 'Recall']
lambda_df = pd.DataFrame(results_lambda)[cols]
print('\\n========== 多任务权重 λ 对弱监督检测性能的影响 ==========')
display(lambda_df)
lambda_df.to_csv('SeqRecon_AD_S_lambda_ablation-v2.csv', index=False, encoding='utf-8-sig')

/tmp/ipykernel_108321/3116511911.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_emb.load_state_dict(torch.load('item_embedding.pt'))



========== λ = 0.8 ==========


Epoch 1/150: 100%|██████████| 112/112 [01:25<00:00,  1.32it/s]


Epoch 1 Completed, Avg Loss: 5.9967
Val AUC: 0.5847, PR-AUC: 0.2593, F1@0.5: 0.0000 | F1@best: 0.3348 (thr=0.160)
Saved new best model (by F1 at optimal threshold)


Epoch 2/150: 100%|██████████| 112/112 [01:24<00:00,  1.33it/s]


Epoch 2 Completed, Avg Loss: 5.2976
Val AUC: 0.6573, PR-AUC: 0.3639, F1@0.5: 0.0000 | F1@best: 0.3379 (thr=0.130)
Saved new best model (by F1 at optimal threshold)


Epoch 3/150: 100%|██████████| 112/112 [01:24<00:00,  1.33it/s]


Epoch 3 Completed, Avg Loss: 5.1767
Val AUC: 0.7122, PR-AUC: 0.3680, F1@0.5: 0.0000 | F1@best: 0.4643 (thr=0.150)
Saved new best model (by F1 at optimal threshold)


Epoch 4/150:  46%|████▌     | 51/112 [00:38<00:46,  1.31it/s]